In [475]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl

def scrape_season(start_year, end_year):

    season_year = {}

    for year in range(start_year,end_year):

        #print('Get the colleges from {year} year'.format(year = year))

        url = 'https://www.sports-reference.com/cbb/seasons/{year}.html'.format(year = str(year))

        # collect HTML data
        html = urlopen(url)
                
        # create beautiful soup object from HTML
        soup = BeautifulSoup(html, features="html")

        # get table
        table = soup.find('table', id='conference-summary')

        # Collecting Data
        conference_data = []
        for row in table.tbody.find_all('tr'):    
            conference_data.append(row)

        rows_data = [[td.getText().replace('-', ' ') for td in conference_data[i].findAll('td')]
                            for i in range(len(conference_data))]

        conference_list =[]

        for conference in rows_data[0:]:
            conference_list.append(conference[0])

        hyperlinks_dict = {}
        for link in table.tbody.find_all('a'):
            if link.get_text() in conference_list:
                hyperlinks_dict[link.get_text()] = link.get('href')

        
        season_dict = {}
        season_dict['season'] = hyperlinks_dict
        season_year[str(year)] = season_dict

    return season_year







In [156]:
start_year = 1987
end_year = 2023

season_links = scrape_season(start_year, end_year)

In [165]:
def find_colleges_links(season_links):

    for year in season_links:

        conference_dict = {}
        for conference in season_links[year]['season']:

            url = season_links[year]['season'][conference]

            # collect HTML data
            root_link = 'https://www.sports-reference.com'
            url = root_link + url
            html = urlopen(url)
                    
            # create beautiful soup object from HTML
            soup = BeautifulSoup(html, features="html")

            # get table
            table = soup.find('table', id='standings')

            # Collecting Data
            college_data = []
            for row in table.tbody.find_all('tr'):    
                college_data.append(row)

            rows_data = [[td.getText().replace('-', ' ') for td in college_data[i].findAll('td')]
                                for i in range(len(college_data))]

            college_list =[]

            for college in rows_data[0:]:
                if(len(college) > 0):
                    college_list.append(college[0])

            hyperlinks_dict = {}
            for link in table.tbody.find_all('a'):
                if link.get_text() in college_list:
                    hyperlinks_dict[link.get_text()] = link.get('href')

            
            conference_dict[conference] = hyperlinks_dict

        season_links[year]['college'] = conference_dict
  

    return season_links

            


In [166]:
global_dict = find_colleges_links(season_links)

In [167]:
global_dict['2021']['college']['Big Ten Conference'].keys()

dict_keys(['Michigan', 'Illinois', 'Iowa', 'Purdue', 'Ohio State', 'Wisconsin', 'Rutgers', 'Maryland', 'Michigan State', 'Indiana', 'Penn State', 'Northwestern', 'Minnesota', 'Nebraska'])

In [168]:
def get_players_links(season_links):

    players_links = []
    for year in season_links:

        print(year)

        
        for conferance in season_links[year]['college']:
            
            
            for college in season_links[year]['college'][conferance]:

                url = season_links[year]['college'][conferance][college]

                # collect HTML data
                root_link = 'https://www.sports-reference.com'
                url = root_link + url
                html = urlopen(url)

                
                        
                # create beautiful soup object from HTML
                soup = BeautifulSoup(html, features="html")

                # get table
                table = soup.find('table', id='roster')

                # Collecting Data
                if table is not None:
                    for row in table.tbody.find_all('tr'):    
                        # Find all data for each column
                        link = row.find_all('a')[0]
                        
                        if link.get('href') not in players_links:
                            players_links.append(link.get('href'))


                else:
                    
                    print('The following link does not have the desirable table ' + url)

                

    return players_links

    

In [169]:
players_links = get_players_links(global_dict)

1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/brown/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/columbia/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/cornell/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/dartmouth/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/harvard/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/pennsylvania/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/princeton/2021.html
The f

In [171]:
players_links[0:5]
len(players_links)

54283

In [472]:
def players_links_excel(players_links):

    id_list = []

    for id in range(len(players_links)):
        id_list.append(id)

    links_players_df = pd.DataFrame()
    links_players_df['id'] = id_list
    links_players_df['link'] = players_links

    links_players_df.to_excel("data/links_players_1987-2022.xlsx")



In [476]:
players_links_excel(players_links)

In [468]:
def player_data_college_reference(player_url, id, player_stat_list):

    global_url = 'https://www.sports-reference.com'
    url = global_url + player_url

    html = urlopen(url)

    pointer_3_year = 1987
                        
    # create beautiful soup object from HTML
    soup = BeautifulSoup(html, features="html")

    #Get player info
    info = soup.find(id = 'info')
    player_name = info.find_all('span')[0].get_text()
    info_draft = [data.getText() for data in info.find_all('strong')]

    if len([data.get_text().replace('\n', '').replace(' ', '') for index, data in enumerate(info.find_all('p')[0])]) > 2:
        info_general = [data.get_text().replace('\n', '').replace(' ', '') for index, data in enumerate(info.find_all('p')[0])]
    else:
        info_general = [data.get_text().replace('\n', '').replace(' ', '') for index, data in enumerate(info.find_all('p')[1])]

    position_index = info_general.index('Position:') + 1
    heigth_index = position_index + 1
    position = info_general[position_index]
    height_cm = info_general[heigth_index].split('(')[1].split(',')[0].replace('cm', '')[0:3]
    height_feet = info_general[heigth_index].split()[0].replace(',', '')
   
    
    if 'Draft:' in info_draft:
        draft = 1
    else:
        draft = 0

    


    # get table
    table = soup.find('table', id='players_per_game')

    # Collecting Data
    per_game_data = []
    for row in table.tbody.find_all('tr'):    
        per_game_data.append(row)
    
    for i in range(len(per_game_data)):
        get_season_year = int(per_game_data[i].find_all('a')[0].get_text().split('-')[0][0:2] + per_game_data[i].find_all('a')[0].get_text().split('-')[1])
        if get_season_year >= pointer_3_year:
            general_data = [id, player_name, position, height_cm, height_feet, draft, get_season_year]
            stats_data = [td.getText() for td in per_game_data[i].findAll('td')]
            player_stat_list.append(general_data + stats_data)

    return player_stat_list



In [469]:
def get_excel_raw_college(players_links):

    per_game_headers = ['id', 'name', 'position', 'heigth_cm', 'heigth_feet', 'draft',
                        'season', 'school', 'conf', 'games', 'games_started', 'mp', 'fg', 'fga', 'fg%',
                        '2p', '2pa', '2p%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb', 'drb', 'tbr', 
                        'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'unknown', 'strength_of_schedule']

    players_stat_list = []

    for id, link in enumerate(players_links):
        try:

            players_stat_list = player_data_college_reference(link, id, players_stat_list)
        except:
            print(id)
    print('Good')

    per_game_df =pd.DataFrame(players_stat_list, columns = per_game_headers)


    return per_game_df

In [470]:
per_game_df = get_excel_raw_college(players_links[0:20])

6-3, 170lb (190cm, 77kg)
6-9, 247lb (206cm, 112kg)
6-11, 230lb (211cm, 104kg)
6-2, 180lb (188cm, 81kg)
6-10, 220lb (208cm, 99kg)
6-1 (185cm)
6-10, 230lb (208cm, 104kg)
6-5 (196cm)
6-6, 215lb (198cm, 97kg)
6-11 (211cm)
6-4 (193cm)
6-2 (188cm)
5-8 (173cm)
6-10, 215lb (208cm, 97kg)
6-2 (188cm)
6-7 (201cm)
6-7 (201cm)
6-11, 215lb (211cm, 97kg)
6-4 (193cm)
6-3 (190cm)
Good


In [471]:
per_game_df

,id,name,position,heigth_cm,heigth_feet,draft,season,school,conf,games,...,drb,tbr,ast,stl,blk,tov,pf,pts,unknown,strength_of_schedule
0,0,Kenny Smith,Guard,190,6-3,1,1987,UNC,ACC,34,...,,2.2,6.1,1.5,0.1,3.1,1.4,16.9,,8.71
1,1,J.R. Reid Jr,Forward,206,6-9,1,1987,UNC,ACC,36,...,,7.4,1.8,1.3,0.8,2.6,3.1,14.7,,8.71
2,1,J.R. Reid Jr,Forward,206,6-9,1,1988,UNC,ACC,33,...,,8.9,1.7,1.2,1.2,3.8,3.4,18.0,,10.72
3,1,J.R. Reid Jr,Forward,206,6-9,1,1989,UNC,ACC,27,...,,6.3,1.3,0.9,0.8,2.6,3.0,15.9,,12.41
4,2,Joe Wolf,Forward,211,6-11,1,1987,UNC,ACC,34,...,,7.1,2.9,1.3,0.3,1.7,2.5,15.2,,8.71
5,3,Jeff Lebo,Guard,188,6-2,0,1987,UNC,ACC,34,...,,2.1,4.2,1.4,0.1,1.9,2.1,13.5,,8.71
6,3,Jeff Lebo,Guard,188,6-2,0,1988,UNC,ACC,33,...,,2.6,4.8,1.0,0.0,2.3,2.0,12.2,,10.72
7,3,Jeff Lebo,Guard,188,6-2,0,1989,UNC,ACC,32,...,,2.6,4.4,1.2,0.1,2.4,2.3,12.2,,12.41
8,4,Dave Popson,Center,208,6-10,1,1987,UNC,ACC,36,...,,4.8,1.3,0.7,0.8,1.4,2.7,10.0,,8.71
9,5,Ranzino Smith,Guard,185,6-1,0,1987,UNC,ACC,36,...,,1.4,1.6,0.8,0.0,1.1,0.9,7.5,,8.71


In [459]:
players_links[5]

'/cbb/players/ranzino-smith-1.html'